<a href="https://colab.research.google.com/github/bintangnabiil/Deep_Learning/blob/main/Deep_Learning_Cifar10_Week_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [47]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve
import numpy as np
import matplotlib.pyplot as plt

def get_pytorch_dataloader(batch_size=64):
    transform = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(10),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616))
    ])
    trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
    testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
    trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True)
    testloader = DataLoader(testset, batch_size=batch_size, shuffle=False)
    return trainloader, testloader

def get_tensorflow_dataset(batch_size=64):
    (train_ds, test_ds), ds_info = tfds.load(
        "cifar10",
        split=["train", "test"],
        as_supervised=True,
        with_info=True,
    )
    def normalize_img(image, label):
        return tf.image.per_image_standardization(image), label
    train_ds = train_ds.map(normalize_img).batch(batch_size).shuffle(10000)
    test_ds = test_ds.map(normalize_img).batch(batch_size)
    return train_ds, test_ds


Model CNN dengan Pytorch

In [48]:
class CNN_PyTorch(nn.Module):
    def __init__(self):
        super(CNN_PyTorch, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(256 * 4 * 4, 512)
        self.fc2 = nn.Linear(512, 10)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)
        self.batchnorm1 = nn.BatchNorm2d(64)
        self.batchnorm2 = nn.BatchNorm2d(128)
        self.batchnorm3 = nn.BatchNorm2d(256)

    def forward(self, x):
        x = self.pool(self.relu(self.batchnorm1(self.conv1(x))))
        x = self.pool(self.relu(self.batchnorm2(self.conv2(x))))
        x = self.pool(self.relu(self.batchnorm3(self.conv3(x))))
        x = x.view(x.size(0), -1)
        x = self.dropout(self.relu(self.fc1(x)))
        x = self.fc2(x)
        return x

Model CNN dengan Tensorflow

In [49]:
def CNN_TensorFlow():
    model = keras.Sequential([
        layers.Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(32, 32, 3)),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(512, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(10, activation='softmax')
    ])
    return model


Model MLP dengan Pytorch

In [50]:
class MLP_PyTorch(nn.Module):
    def __init__(self):
        super(MLP_PyTorch, self).__init__()
        self.fc1 = nn.Linear(32 * 32 * 3, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 10)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

Model MLP dengan Tensorflow

In [51]:
def MLP_TensorFlow():
    model = keras.Sequential([
        layers.Flatten(input_shape=(32, 32, 3)),
        layers.Dense(512, activation='relu'),
        layers.Dense(256, activation='relu'),
        layers.Dense(10, activation='softmax')
    ])
    return model

Inisialisasi Model

In [52]:
cnn_pytorch = CNN_PyTorch()
mlp_pytorch = MLP_PyTorch()
cnn_tf = CNN_TensorFlow()
mlp_tf = MLP_TensorFlow()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model Evaluasi Akurasi, Presisi, Recall, F1, dan AUC

In [53]:
def evaluate_pytorch_model(model, test_loader, device):
    model.eval()
    y_true, y_pred = [], []
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(predicted.cpu().numpy())

    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='macro')
    recall = recall_score(y_true, y_pred, average='macro')
    f1 = f1_score(y_true, y_pred, average='macro')
    auc = roc_auc_score(y_true, np.eye(10)[y_pred], multi_class='ovr')

    print(f'Accuracy: {accuracy:.4f}')
    print(f'Precision: {precision:.4f}')
    print(f'Recall: {recall:.4f}')
    print(f'F1 Score: {f1:.4f}')
    print(f'AUC: {auc:.4f}')

    # Tentukan perangkat (GPU/CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
cnn_pytorch.to(device)

optimizer = optim.Adam(cnn_pytorch.parameters(), lr=0.0005, weight_decay=1e-4)
transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616))
])
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

# Load DataLoader
_, test_loader = get_pytorch_dataloader()

# Evaluasi model CNN PyTorch
evaluate_pytorch_model(cnn_pytorch, test_loader, device)

Accuracy: 0.1078
Precision: 0.0696
Recall: 0.1078
F1 Score: 0.0745
AUC: 0.5043


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Summary Model Untuk Pytorch


In [43]:
cnn_tf.summary()
mlp_tf.summary()

# Untuk PyTorch
print(cnn_pytorch)
print(mlp_pytorch)

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_8 (Conv2D)                    │ (None, 32, 32, 64)          │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_6                │ (None, 32, 32, 64)          │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_8 (MaxPooling2D)       │ (None, 16, 16, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_9 (Conv2D)                    │ (None, 16, 16, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_7                │ (None, 16, 16, 128)         │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_9 (MaxPooling2D)       │ (None, 8, 8, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_10 (Conv2D)                   │ (None, 8, 8, 256)           │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_8                │ (None, 8, 8, 256)           │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_10 (MaxPooling2D)      │ (None, 4, 4, 256)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_6 (Flatten)                  │ (None, 4096)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_15 (Dense)                     │ (None, 512)                 │       2,097,664 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_16 (Dense)                     │ (None, 10)                  │           5,130 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,475,402 (9.44 MB)

 Trainable params: 2,474,506 (9.44 MB)

 Non-trainable params: 896 (3.50 KB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ flatten_7 (Flatten)                  │ (None, 3072)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_17 (Dense)                     │ (None, 512)                 │       1,573,376 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_18 (Dense)                     │ (None, 256)                 │         131,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_19 (Dense)                     │ (None, 10)                  │           2,570 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,707,274 (6.51 MB)

 Trainable params: 1,707,274 (6.51 MB)

 Non-trainable params: 0 (0.00 B)

CNN_PyTorch(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=4096, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=10, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.5, inplace=False)
  (batchnorm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)
MLP_PyTorch(
  (fc1): Linear(in_features=3072, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (fc3): Linear(in_features=256, out_features=10, bias=

#Persamaan Matematika

##1. Fungsi Aktivasi ReLU
Digunakan dalam model CNN dan MLP baik untuk PyTorch maupun TensorFlow. ReLU (Rectified Linear Unit) didefinisikan sebagai:

$f(x)=max(0,x)$

ReLU menghilangkan nilai negatif dengan menggantinya dengan nol, sementara nilai positif tetap dipertahankan. Ini membantu dalam mengatasi masalah vanishing gradient pada model deep learning.<br><br>

##2. Operasi Konvolusi (CNN)
Lapisan konvolusi digunakan untuk mengekstrak fitur dari gambar. Operasi konvolusi didefinisikan sebagai:

$O(i,j) = \sum_m\sum_nI(i+m,j+n)\times K(m,n)$

Di mana:
- $O(i,j)$ = output di posisi (i,j)
- $I(i+m,j+n)$ = input gambar,
- $K(m,n)$ = kernel/filter konvolusi.

Dalam model, terdapat dua lapisan konvolusi:
- Lapisan Pertama: 3 kanal (RGB) --> 32 filter dengan ukuran 3 $\times$ 3
- Lapisan Kedua: 32 kanal (RGB) --> 64 filter dengan ukuran 3 $\times$ 3<br><br>

##3. Pooling (MaxPooling 2x2)
Pooling bertujuan untuk mengurangi dimensi feature map tanpa menghilangkan informasi penting.

$O(i,j) = \underset{\text{(m,n)∈P}}{\text{max}}I(i+m,j+n)$

Pooling dilakukan dengan window 2 $\times$ 2 dan stride 2, sehingga ukuran gambar diperkecil setengahnya.<br><br>

##4. Fungsi Softmax (Klasifikasi Multikelas)
Digunakan pada lapisan output untuk mengubah nilai output menjadi probabilitas:

$σ(z)_i = \frac{e^{zi}}{\sum_{j=1}^Ne^{zj}}$

Di mana:
- $z_i$ = skor keluaran untuk kelas $i$,
- $N$ =  jumlah kelas (10 pada CIFAR-10).

Softmax memastikan bahwa probabilitas total dari semua kelas selalu berjumlah 1.<br><br>

##5. Fungsi Kehilangan (Cross-Entropy Loss)
Digunakan untuk mengukur seberapa baik model memprediksi label. Rumusnya:

$L = -\sum\limits_{i=1}^{N} y_i log(\hat{y}_i)$

Di mana:
- $y_i$ =  label sebenarnya (one-hot encoded),
- $\hat{y}_i$ = probabilitas yang diprediksi oleh model untuk kelas $i$.

Cross-entropy sangat efektif untuk klasifikasi multikelas seperti CIFAR-10.<br><br>

##6. Backward Propagation dan Optimasi (Stochastic Gradient Descent - SGD)
Model diperbarui dengan menggunakan aturan pembaruan bobot:

$w_{t+1}=w_t-η∇L$

Di mana:
- $w_t$ = bobot pada iterasi ke-$t$,
- $η$ = laju pembelajaran (learning rate),
- $∇L$ = gradien dari loss function terhadap bobot.

Optimasi ini membantu model belajar untuk meminimalkan loss.<br><br>

##7. Evaluasi Model
###a) Akurasi

$Accuracy = \frac{Jumlah Prediksi Benar}{Total Data}$<br><br>

###b) Presisi

$Precision = \frac{TP}{TP+FP}$

Menunjukkan seberapa banyak prediksi positif yang benar dari seluruh prediksi positif yang diberikan oleh model.<br><br>

###c) Recall (Sensitivitas)

$Recall = \frac{TP}{TP+FN}$

Menunjukkan seberapa banyak data positif yang berhasil diklasifikasikan dengan benar oleh model.<br><br>

###d) F1-Score

$F1=2\times\frac{Precision \times Recall}{Precision+Recall}$

Metrik yang menyeimbangkan antara Precision dan Recall. <br><br>

###e) AUC-ROC
AUC (Area Under Curve) ROC (Receiver Operating Characteristic) mengukur seberapa baik model membedakan antara kelas-kelas dengan membandingkan nilai probabilitas yang diberikan oleh softmax.